## Initial imports

In [1]:
import os
import math
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
import pycocotools
from torch.utils.tensorboard import SummaryWriter

In [2]:
# Code from Maastricht to access GPU of choice

import torch
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [3]:
torch.cuda.set_device(0)
device = torch.device(f'cuda:{0}' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.cuda.current_device(), torch.cuda.get_device_name()

(0, 'GeForce GTX 1080 Ti')

## Load data

In [5]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode


# Class for a customized dataset
# In this case preprocessed CEM images combined in a 3-channel RGB .jpg format
# and the corresponding mask of present lesions in a 1-channel .png format
class CustomImageDataset(Dataset):
    def __init__(self, root, annotations_file, img_dir, mask_dir, transform=None, target_transform=None):
        # Read the .csv file with all the information
        self.img_labels = pd.read_csv(os.path.join(root, annotations_file))
        # Define the directories of the images and masks
        self.img_dir = os.path.join(root, img_dir)
        self.mask_dir = os.path.join(root, mask_dir)
        # Define whethet transformations are included
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        # Return the number of cases in the dataset
        # In this set, CC and MLO of the same breast are considered different cases
        return len(self.img_labels)

    def __getitem__(self, idx):
        # Read the image and the mask for a case from the directories
        img_path = self.img_labels.iloc[idx, 0]
        mask_path = self.img_labels.iloc[idx,6]
        img_path = img_path.replace('E:', '\\\\tsclient\E')
        mask_path = mask_path.replace('E:', '\\\\tsclient\E')
        image = read_image(img_path).float()
        mask = read_image(mask_path)

        # Resize so all images and masks have the same size
#         image = T.Resize([800,800])(image)
#         mask = T.Resize([800,800])(mask)    
#         resize_scale_x = 800/image.size()[1]
#         resize_scale_y = 800/image.size()[2]

        # Resize if necessary
        # First the smallest dimension is reduced to 400 if it is larger
        # Then the largest dimension is reduced to 650 if it is still larger
        resize_scale = 1.0
        min_size_idx = np.argmin([image.size()[1], image.size()[2]])
        
        if min_size_idx == 0 and image.size()[1] > 400 :
            resize_scale *= 400/image.size()[1]
            image = T.Resize([400, int(400*image.size()[2]/image.size()[1])])(image)
            mask = T.Resize([400, int(400*image.size()[2]/image.size()[1])], interpolation=InterpolationMode.NEAREST)(mask)
        if min_size_idx == 0 and image.size()[2] > 650 :
            resize_scale *= 650/image.size()[2]
            image = T.Resize([int(650*image.size()[1]/image.size()[2]), 650])(image)
            mask = T.Resize([int(650*image.size()[1]/image.size()[2]), 650], interpolation=InterpolationMode.NEAREST)(mask)
        
        if min_size_idx == 1 and image.size()[2] > 400 :
            resize_scale *= 400/image.size()[2]
            image = T.Resize([int(400*image.size()[1]/image.size()[2]), 400])(image)
            mask = T.Resize([int(400*image.size()[1]/image.size()[2]), 400], interpolation=InterpolationMode.NEAREST)(mask)
        if min_size_idx == 1 and image.size()[1] > 650 :
            resize_scale *= 650/image.size()[1]
            image = T.Resize([650, int(650*image.size()[2]/image.size()[1])])(image)
            mask = T.Resize([650, int(650*image.size()[2]/image.size()[1])], interpolation=InterpolationMode.NEAREST)(mask)
            
        # Normalize image with mean and standard deviation per channel
        mean = torch.mean(image, dim=(1,2))
        stdev = torch.std(image, dim=(1,2))
        image = T.Normalize(mean, stdev)(image)
        
        # Rescale to [0,1] range per channel
        for dim in range(3) :
            image[dim] -= torch.min(image[dim])
            image[dim] /= torch.max(image[dim])  
        
        # Create separate channel in mask for each lesion
        mask_out = np.zeros((torch.max(mask).item(), mask.shape[-2], mask.shape[-1]))
        for lesion_idx in range(torch.max(mask).item()) :
#             mask_out[lesion_idx][mask[0]==lesion_idx+1] = 1  
            mask_out[lesion_idx][mask[0]>0] = 1 # alternative for wrong masks with only one lesion
        
        # Read the location of the lesion bounding box from the .csv file
        xmin = self.img_labels.iloc[idx, 1]
        xmax = self.img_labels.iloc[idx, 2]
        ymin = self.img_labels.iloc[idx, 3]
        ymax = self.img_labels.iloc[idx, 4]
        boxes = [[ymin*resize_scale, xmin*resize_scale, ymax*resize_scale, xmax*resize_scale]]
        # Read the label of the lesion from the .csv file
        labels = self.img_labels.iloc[idx, 5]  
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)

        labels = torch.tensor([labels])
        image_id = torch.tensor([idx])
        image_name = img_path[-20:]
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        mask_out = torch.from_numpy(mask_out)
        mask_out = mask_out.to(torch.uint8)
        
        iscrowd = torch.zeros((2,), dtype=torch.int64)
        
        # Apply transformations if defined
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)  
            
        target = {}
        target["image_id"] = image_id
        target["image_name"] = image_name
        target["masks"] = mask_out
        target["boxes"] = boxes
        target["area"] = area
        target["labels"] = labels
        target["iscrowd"] = iscrowd
            
        return image, target


In [12]:
# testdatadir = 'B:\\Astrid\\Preprocessed\\TrainCalcClusterSynthetic'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_synthetic_onlymaldef.csv', 'colored_to_jpg', 'mask_to_png')
# test_data = CustomImageDataset(testdatadir, 'annotations_val_calccluster_synthetic_onlymaldef.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TrainCalcClusterReal'
# test_data = CustomImageDataset(testdatadir, 'annotations_train_calccluster_real_onlymal.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TestCalcClusterSynthetic_SameImage'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_synthetic.csv', 'colored_to_jpg', 'mask_to_png')

# testdatadir = 'B:\\Astrid\\Preprocessed\\TestCalcClusterSynthetic_SameCluster'
# test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_synthetic.csv', 'colored_to_jpg', 'mask_to_png')

testdatadir = 'B:\\Astrid\\Preprocessed\\TestCalcClusterReal'
test_data = CustomImageDataset(testdatadir, 'annotations_test_calccluster_real_onlymal.csv', 'colored_to_jpg', 'mask_to_png')
test_data_short = []
for idx in range(16) :
    test_data_short.append(test_data[idx])

savedir = 'B:\\Astrid\\Preprocessed\\Models'

In [13]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import utils
from torch.utils.data import random_split

# Read test dataset
# test_dataloader = DataLoader(test_data, batch_size=4, shuffle=False, collate_fn=utils.collate_fn)
test_dataloader = DataLoader(test_data_short, batch_size=4, shuffle=False, collate_fn=utils.collate_fn)


In [14]:
len(test_data)

76

In [15]:
len(test_data_short)

16

## Load model

In [16]:
import torchvision
from torchvision.models.detection import FasterRCNN, MaskRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

mask_roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                     output_size=14,
                                                     sampling_ratio=2)


# put the pieces together inside a MaskRCNN model
model = MaskRCNN(backbone,
                 num_classes=3,
                 rpn_anchor_generator=anchor_generator,
                 box_roi_pool=roi_pooler,
                 mask_roi_pool=mask_roi_pooler,
                 min_size=400,
                 max_size=650)


In [17]:
model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(400,), max_size=650, mode='bilinear')
  )
  (backbone): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
   

In [18]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9)

In [28]:
## Load from dict
# model.load_state_dict(torch.load(os.path.join(savedir, 'dict_malsynth.dict')))

## Load from checkpoint
checkpoint = torch.load(os.path.join('B:\\Astrid\\Preprocessed\\Models', 'model_maslsynth200_100.pth'))
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
# loss = checkpoint['loss']

model.eval()

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(400,), max_size=650, mode='bilinear')
  )
  (backbone): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
   

In [17]:
iter_data = iter(test_dataloader)

already_used = 0
used_now = 0
while used_now < already_used :
    test_images, test_targets = next(iter_data)
    used_now += 1

In [18]:
# For Testing
test_images, test_targets = next(iter_data)
test_image_list = list(image for image in test_images)
test_target_list = [{k: v for k, v in t.items()} for t in test_targets]

In [19]:
test_images_cuda = []
for te in test_images :
    test_images_cuda.append(te.to(device))

In [20]:
predictions = model(test_images_cuda)

In [21]:
predictions

[{'boxes': tensor([[187.1421, 402.9792, 206.5778, 422.5407],
          [182.6679, 411.9633, 193.4831, 419.4932],
          [187.5260, 379.5725, 201.7404, 391.8248],
          [200.6170, 410.8705, 212.5491, 420.3547],
          [193.8357, 367.6738, 227.6961, 458.2506],
          [188.1134, 429.3427, 201.5413, 444.8321],
          [207.6683, 409.7120, 219.7329, 418.8199]], device='cuda:0',
         grad_fn=<StackBackward0>),
  'labels': tensor([1, 1, 1, 1, 1, 1, 1], device='cuda:0'),
  'scores': tensor([0.8124, 0.7615, 0.6263, 0.5983, 0.4497, 0.3151, 0.1199],
         device='cuda:0', grad_fn=<IndexBackward0>),
  'masks': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            ...,
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]]],
  
  
          [[[0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.

In [22]:
test_target_list

[{'image_id': tensor([0]),
  'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
  'boxes': tensor([[110.5079, 471.9824, 132.1964, 491.2610]]),
  'area': tensor([418.1237]),
  'labels': tensor([1]),
  'iscrowd': tensor([0, 0])},
 {'image_id': tensor([1]),
  'masks': tensor([[[0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           ...,
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0],
           [0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0],

In [13]:
device_cpu = torch.device('cpu')

In [23]:
for pred_idx, pred in enumerate(predictions) :
    
    target_box = test_target_list[pred_idx]['boxes']
    target_area = test_target_list[pred_idx]['area']
    target_id = test_target_list[pred_idx]['image_id']
    
    for box_idx, pred_box in enumerate(pred['boxes']) :
        pred_score = pred['scores'][box_idx]
        pred_area = (pred_box[2].to(device_cpu)-pred_box[0].to(device_cpu))*(pred_box[3].to(device_cpu)-pred_box[1].to(device_cpu))
            
#         print(box_idx, target_box, pred_box, pred_score)
        
        xs = max(target_box[0][0],pred_box[0].to(device_cpu))
        ys = max(target_box[0][1],pred_box[1].to(device_cpu))
        xe = min(target_box[0][2],pred_box[2].to(device_cpu))
        ye = min(target_box[0][3],pred_box[3].to(device_cpu))
        if xe-xs > 0 and ye-ys > 0 :
            overlap_area = (xe-xs)*(ye-ys)
        else :
            overlap_area = -1
        
        if pred_score > 0.4 and overlap_area > 0.5*target_area :
            print('Lesion detected', target_id, pred_score, overlap_area)

Lesion detected tensor([1]) tensor(0.4126, device='cuda:0', grad_fn=<SelectBackward0>) tensor(133.0899, grad_fn=<MulBackward0>)


In [16]:
import time

def evaluate_kulum(model, data_loader, device):
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    cpu_device = torch.device("cpu")
    model.eval()
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    evaluator_time = time.time()
    
    detected_list = []
    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)

        for pred_idx, pred in enumerate(outputs) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']

            for box_idx, pred_box in enumerate(pred['boxes']) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    overlap_area = (xe-xs)*(ye-ys)
                else :
                    overlap_area = -1

                if pred_score > 0.4 and overlap_area > 0.25*target_area :
                    print('Lesion detected', target_id, pred_score, overlap_area)
                    detected_list.append([target_id, pred_score, overlap_area])
        

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)

    torch.set_num_threads(n_threads)
    torch.cuda.empty_cache()
    
    return metric_logger, detected_list

In [17]:
det_list = evaluate_kulum(model, test_dataloader, device)


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 11.00 GiB total capacity; 8.30 GiB already allocated; 12.32 MiB free; 8.44 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
det_list

NameError: name 'det_list' is not defined

In [15]:
torch.cuda.empty_cache()

In [29]:
from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset
import time

def _get_iou_types(model):
    model_without_ddp = model
    if isinstance(model, torch.nn.parallel.DistributedDataParallel):
        model_without_ddp = model.module
    iou_types = ["bbox"]
    if isinstance(model_without_ddp, torchvision.models.detection.MaskRCNN):
        iou_types.append("segm")
    if isinstance(model_without_ddp, torchvision.models.detection.KeypointRCNN):
        iou_types.append("keypoints")
    return iou_types

def evaluate(model, data_loader, device):
    
    n_threads = torch.get_num_threads()
    # FIXME remove this and make paste_masks_in_image run on the GPU
    torch.set_num_threads(1)
    
    cpu_device = torch.device("cpu")
    
    model.eval()
    
    metric_logger = utils.MetricLogger(delimiter="  ")
    header = "Test:"

    coco = get_coco_api_from_dataset(data_loader.dataset)
    iou_types = _get_iou_types(model)
    coco_evaluator = CocoEvaluator(coco, iou_types)

#     detected_list = []
    for images, targets in metric_logger.log_every(data_loader, 100, header):
        images = list(img.to(device) for img in images)

        if torch.cuda.is_available():
            torch.cuda.synchronize()
        model_time = time.time()
        outputs = model(images)

        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        model_time = time.time() - model_time

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        evaluator_time = time.time()
        coco_evaluator.update(res)
        evaluator_time = time.time() - evaluator_time
        metric_logger.update(model_time=model_time, evaluator_time=evaluator_time)
        
        for pred_idx, pred in enumerate(outputs) :

            target_box = targets[pred_idx]['boxes']
            target_area = targets[pred_idx]['area']
            target_id = targets[pred_idx]['image_id']
            target_name = targets[pred_idx]['image_name']

            for box_idx, pred_box in enumerate(pred['boxes']) :
                pred_score = pred['scores'][box_idx]
                pred_area = (pred_box[2]-pred_box[0])*(pred_box[3]-pred_box[1])

                xs = max(target_box[0][0],pred_box[0])
                ys = max(target_box[0][1],pred_box[1])
                xe = min(target_box[0][2],pred_box[2])
                ye = min(target_box[0][3],pred_box[3])
                if xe-xs > 0 and ye-ys > 0 :
                    overlap_area = (xe-xs)*(ye-ys)
                else :
                    overlap_area = -1

                if pred_score > 0.1 and overlap_area > 0.1*target_area :
                    print('Lesion detected', target_id, target_name, pred_score, overlap_area)
#                     detected_list.append([target_id, pred_score, overlap_area])
        

    # gather the stats from all processes
    metric_logger.synchronize_between_processes()
    print("Averaged stats:", metric_logger)
    coco_evaluator.synchronize_between_processes()

    # accumulate predictions from all images
    coco_evaluator.accumulate()
    coco_evaluator.summarize()
    torch.set_num_threads(n_threads)
    return coco_evaluator

In [30]:
cocoev = evaluate(model, test_dataloader, device=device)

creating index...
index created!
Test:  [0/4]  eta: 0:00:00  model_time: 0.2190 (0.2190)  evaluator_time: 0.0190 (0.0190)  time: 0.2480  data: 0.0020  max mem: 7545
Test:  [3/4]  eta: 0:00:00  model_time: 0.1830 (0.1907)  evaluator_time: 0.0060 (0.0145)  time: 0.2145  data: 0.0010  max mem: 7717
Test: Total time: 0:00:00 (0.2150 s / it)
Averaged stats: model_time: 0.1830 (0.1907)  evaluator_time: 0.0060 (0.0145)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets